### Interpolate Velocities
Read and interpolate time-variable ice velocity grids to fill data gaps

In [ ]:
import os
import numpy as np
import xarray as xr
import pointAdvection
import pointCollection as pc
import matplotlib.pyplot as plt
import matplotlib.colors as colors

#### Read input velocity field using ``xarray`` and ``pointAdvection``

In [ ]:
vel_dir='/Volumes/ice1/ben/ArcticDEM/fullres/Nodenskold/velocity'

# read velocity file with xarray
with xr.open_dataset(os.path.join(vel_dir,'Norden.NSIDC-0731.nc')) as ds:
    dt=np.array(ds.time.data-np.datetime64('2001-01-01'),dtype='timedelta64[s]')
    adv=pointAdvection.advection().from_dict({'x':np.array(ds.x),
                                'y':np.array(ds.y), 
                                'U':np.array(ds.VelocitySeries[:,0,::-1,:]),
                                'V':np.array(ds.VelocitySeries[:,1,::-1,:]), 
                                'eU':np.array(ds.VelocitySeries[:,3,::-1,:]),
                                'eV':np.array(ds.VelocitySeries[:,4,::-1,:]),
                                'time':dt}, t_axis=0, scale=1.0)

#### Plot original velocity with data gaps

In [ ]:
# create output figure axis
fig,ax = plt.subplots(num=1, ncols=2, sharex=True, sharey=True, figsize=(12,6))
# create color map
cmap = pointAdvection.tools.custom_colormap(180, 'Rignot')
# show velocity magnitude
adv.imshow(ax=ax[0], band=1, cmap=cmap, norm=colors.LogNorm(vmin=1, vmax=3000), alpha=0.75)
# show velocity divergence
adv.imshow(ax=ax[1], band=1, imtype='divergence', cmap=plt.cm.coolwarm, vmin=-0.1, vmax=0.1, alpha=0.75)
# tight layout
fig.tight_layout()
plt.show()

#### Calculate an error-weighted version to fill data gaps

In [ ]:
# calculate an error-weighted average of the velocities
v=adv.velocity
wU=1/v.eU**2/np.nansum(1/v.eU**2, axis=2)[:,:,None]
wV=1/v.eV**2/np.nansum(1/v.eV**2, axis=2)[:,:,None]
vbar=pc.grid.data().from_dict({'x':np.array(v.x),
                            'y':np.array(v.y), 
                            'U':np.nansum(wU*v.U, axis=2),
                            'V':np.nanmean(wV*v.V, axis=2)})

# attempt to fill in gaps in each velocity field with the average of 
# the velocity from one year prior and that from one year later.
# if one or the other of these is missing, use valid values from the 
# slice that is present.  

v=adv.velocity
v_filled=adv.velocity.copy()

delta_year=np.timedelta64(24*3600*365, 's')
delta_tol = 24*3600*365/8

for ii in range(v.U.shape[2]-1):
    this_U=v.U[:,:,ii].copy()
    this_V=v.V[:,:,ii].copy()
    u_temp = np.zeros_like(this_U)
    v_temp = np.zeros_like(this_U)
    w_temp = np.zeros_like(this_U)
    for dt in [-delta_tol, delta_tol]:
        other_year = np.argmin(np.abs(v.time[ii]+delta_year-v.time))
        if np.abs((v.time[other_year]-v.time[ii]).astype(float)) > delta_tol:
            continue
        good = np.isfinite(v.U[:,:,other_year])
        u_temp[good] += v.U[:,:,other_year][good]
        v_temp[good] += v.V[:,:,other_year][good]
        w_temp[good] += 1
    u_temp[w_temp > 0] /= w_temp[w_temp>0]
    v_temp[w_temp > 0] /= w_temp[w_temp>0]
    to_replace = ((~np.isfinite(this_U)) & (w_temp>0)).ravel()
    if np.any(to_replace):
        this_U.ravel()[to_replace] = u_temp.ravel()[to_replace]
        this_V.ravel()[to_replace] = v_temp.ravel()[to_replace]
    v_filled.U[:,:,ii]=this_U
    v_filled.V[:,:,ii]=this_V

# fill in the remaining gaps using the mean velocity field
    
for ii in range(v.U.shape[2]):
    this_U=v.U[:,:,ii].copy()
    this_V=v.V[:,:,ii].copy()
    to_replace = (~np.isfinite(this_U)).ravel()
    if np.any(to_replace):
        this_U.ravel()[to_replace] = vbar.U.ravel()[to_replace]
        this_V.ravel()[to_replace] = vbar.V.ravel()[to_replace]
    v_filled.U[:,:,ii]=this_U
    v_filled.V[:,:,ii]=this_V

# replace original velocities with filled
adv.velocity.V = v_filled.V
adv.velocity.U = v_filled.U

#### Plot gap-filled velocity fields

In [ ]:
# create output figure axis
fig,ax = plt.subplots(num=2, ncols=2, sharex=True, sharey=True, figsize=(12,6))
# create color map
cmap = pointAdvection.tools.custom_colormap(180, 'Rignot')
# show velocity magnitude
adv.imshow(ax=ax[0], band=1, cmap=cmap, norm=colors.LogNorm(vmin=1, vmax=3000), alpha=0.75)
# show velocity divergence
adv.imshow(ax=ax[1], band=1, imtype='divergence', cmap=plt.cm.coolwarm, vmin=-0.1, vmax=0.1, alpha=0.75)
# tight layout
fig.tight_layout()
plt.show()